Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Mobilenet v2 Quantization with ONNX Runtime on CPU

In this tutorial, we will load a mobilenet v2 model pretrained with [PyTorch](https://pytorch.org/), export the model to ONNX, and then quantize and run with ONNXRuntime.

## 0. Prerequisites ##

If you have Jupyter Notebook, you can run this notebook directly with it. You may need to install or upgrade [PyTorch](https://pytorch.org/), [OnnxRuntime](https://microsoft.github.io/onnxruntime/), and other required packages.

Otherwise, you can setup a new environment. First, install [AnaConda](https://www.anaconda.com/distribution/). Then open an AnaConda prompt window and run the following commands:

```console
conda create -n cpu_env python=3.8
conda activate cpu_env
conda install jupyter
jupyter notebook
```
The last command will launch Jupyter Notebook and we can open this notebook in browser to continue.

### 0.1 Install packages
Let's install nessasary packages to start the tutorial. We will install PyTorch 1.8, OnnxRuntime 1.8, latest ONNX and pillow.

In [13]:
# Install or upgrade PyTorch 1.8.0 and OnnxRuntime 1.8 for CPU-only.
import sys
!{sys.executable} -m pip install --upgrade torch==1.8.0 torchvision==0.9.0 torchaudio===0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
!{sys.executable} -m pip install --upgrade onnxruntime==1.8.0
!{sys.executable} -m pip install --upgrade onnx
!{sys.executable} -m pip install --upgrade pillow

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Requirement already up-to-date: torch==1.8.0 in /Users/gwang/opt/anaconda3/lib/python3.7/site-packages (1.8.0)
Requirement already up-to-date: torchvision==0.9.0 in /Users/gwang/opt/anaconda3/lib/python3.7/site-packages (0.9.0)
Requirement already up-to-date: torchaudio===0.8.0 in /Users/gwang/opt/anaconda3/lib/python3.7/site-packages (0.8.0)
Requirement already up-to-date: onnxruntime==1.8.0 in /Users/gwang/opt/anaconda3/lib/python3.7/site-packages (1.8.0)
Requirement already up-to-date: onnx in /Users/gwang/opt/anaconda3/lib/python3.7/site-packages (1.9.0)
Requirement already up-to-date: pillow in /Users/gwang/opt/anaconda3/lib/python3.7/site-packages (8.2.0)


# 1 Download pretrained model and export to ONNX

In this step, we load a pretrained mobilenet v2 model, and export it to ONNX.

### 1.1 Load the pretrained model
Use torchvision provides API to load mobilenet_v2 model.

In [5]:
from torchvision import models, datasets, transforms as T
mobilenet_v2 = models.mobilenet_v2(pretrained=True)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /Users/gwang/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


### 1.2 Export the model to ONNX
Pytorch onnx export API to export the model.

In [6]:
import torch
image_height = 224
image_width = 224
x = torch.randn(1, 3, image_height, image_width, requires_grad=True)
torch_out = mobilenet_v2(x)

# Export the model
torch.onnx.export(mobilenet_v2,              # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "mobilenet_v2_float.onnx", # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=12,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output']) # the model's output names


### 1.3 Sample Execution with ONNXRuntime

Run an sample with the full precision ONNX model. Firstly, implement the preprocess.

In [7]:
from PIL import Image
import numpy as np
import onnxruntime
import torch

def preprocess_image(image_path, height, width, channels=3):
    image = Image.open(image_path)
    image = image.resize((width, height), Image.ANTIALIAS)
    image_data = np.asarray(image).astype(np.float32)
    image_data = image_data.transpose([2, 0, 1]) # transpose to CHW
    mean = np.array([0.079, 0.05, 0]) + 0.406
    std = np.array([0.005, 0, 0.001]) + 0.224
    for channel in range(image_data.shape[0]):
        image_data[channel, :, :] = (image_data[channel, :, :] / 255 - mean[channel]) / std[channel]
    image_data = np.expand_dims(image_data, 0)
    return image_data

#### Download the imagenet labels and load it

In [8]:
# Download ImageNet labels
!curl -o imagenet_classes.txt https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10472  100 10472    0     0  39817      0 --:--:-- --:--:-- --:--:-- 39817


#### Run the example with ONNXRuntime

In [9]:
session_fp32 = onnxruntime.InferenceSession("mobilenet_v2_float.onnx")

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def run_sample(session, image_file, categories):
    output = session.run([], {'input':preprocess_image(image_file, image_height, image_width)})[0]
    output = output.flatten()
    output = softmax(output) # this is optional
    top5_catid = np.argsort(-output)[:5]
    for catid in top5_catid:
        print(categories[catid], output[catid])

run_sample(session_fp32, 'cat.jpg', categories)

tabby 0.7289727
tiger cat 0.12759951
Egyptian cat 0.12654182
tiger 0.0031891621
lynx 0.0031400118


# 2 Quantize the model with ONNXRuntime 
In this step, we load the full precison model, and quantize it with ONNXRuntime quantization tool. And show the model size comparison between full precision and quantized model. Finally, we run the same sample with the quantized model

## 2.1 Implement a CalibrationDataReader
CalibrationDataReader takes in calibration data and generates input for the model

In [10]:
from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantType
import os

def preprocess_func(images_folder, height, width, size_limit=0):
    image_names = os.listdir(images_folder)
    if size_limit > 0 and len(image_names) >= size_limit:
        batch_filenames = [image_names[i] for i in range(size_limit)]
    else:
        batch_filenames = image_names
    unconcatenated_batch_data = []

    for image_name in batch_filenames:
        image_filepath = images_folder + '/' + image_name
        image_data = preprocess_image(image_filepath, height, width)
        unconcatenated_batch_data.append(image_data)
    batch_data = np.concatenate(np.expand_dims(unconcatenated_batch_data, axis=0), axis=0)
    return batch_data


class MobilenetDataReader(CalibrationDataReader):
    def __init__(self, calibration_image_folder):
        self.image_folder = calibration_image_folder
        self.preprocess_flag = True
        self.enum_data_dicts = []
        self.datasize = 0

    def get_next(self):
        if self.preprocess_flag:
            self.preprocess_flag = False
            nhwc_data_list = preprocess_func(self.image_folder, image_height, image_width, size_limit=0)
            self.datasize = len(nhwc_data_list)
            self.enum_data_dicts = iter([{'input': nhwc_data} for nhwc_data in nhwc_data_list])
        return next(self.enum_data_dicts, None)

## 2.2 Quantize the model

As we can not upload full calibration data set for copy right issue, we only demenstrate with some example images. You need to use your own calibration in practice.

In [35]:
# change it to your real calibration data set
calibration_data_folder = "calibration_imagenet"
dr = MobilenetDataReader(calibration_data_folder)

quantize_static('mobilenet_v2_float.onnx',
                'mobilenet_v2_uint8.onnx',
                dr)

print('ONNX full precision model size (MB):', os.path.getsize("mobilenet_v2_float.onnx")/(1024*1024))
print('ONNX quantized model size (MB):', os.path.getsize("mobilenet_v2_uint8.onnx")/(1024*1024))

2021-06-03 18:16:55,965 PIL.PngImagePlugin [DEBUG] - STREAM b'IHDR' 16 13
2021-06-03 18:16:55,966 PIL.PngImagePlugin [DEBUG] - STREAM b'IDAT' 41 65536


ONNX full precision model size (MB): 13.31911563873291
ONNX quantized model size (MB): 3.4079113006591797


## 2.3 Run the model with OnnxRuntime

In [36]:
session_quant = onnxruntime.InferenceSession("mobilenet_v2.uint8.onnx")
run_sample(session_quant, 'cat.jpg', categories)

tabby 0.8141141
tiger cat 0.11124368
Egyptian cat 0.06763518
lynx 0.0012628529
tiger 0.0012628529


# 3 Convert the models to ort format.

This step is optional, we will convert the *mobilenet_v2_float.onnx* and *mobilenet_v2_uint8.onnx* to ORT format, to be used in mobile applications.

In [38]:
!{sys.executable} -m onnxruntime.tools.convert_onnx_models_to_ort --optimization_level basic ./

2021-06-03 17:27:58.225093 [E:onnxruntime:Default, provider_bridge_ort.cc:902 Ensure] Failed to load library libonnxruntime_providers_shared.dylib with error: dlopen(libonnxruntime_providers_shared.dylib, 10): image not found
2021-06-03 17:27:58.225121 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:1615 pybind11_init_onnxruntime_pybind11_state] Init provider bridge failed.
Converting optimized ONNX model /Users/gwang/github/onnxruntime3/onnxruntime/python/tools/quantization/notebooks/imagenet_v2/mobilenet_v2_float.onnx to ORT format model /Users/gwang/github/onnxruntime3/onnxruntime/python/tools/quantization/notebooks/imagenet_v2/mobilenet_v2_float.basic.ort
Converting optimized ONNX model /Users/gwang/github/onnxruntime3/onnxruntime/python/tools/quantization/notebooks/imagenet_v2/mobilenet_v2_float-opt.onnx to ORT format model /Users/gwang/github/onnxruntime3/onnxruntime/python/tools/quantization/notebooks/imagenet_v2/mobilenet_v2_float-opt.basic.ort
Converting optimized ONNX mod

Converted 4 models. 0 failures.
2021-06-03 18:27:58,593 ort_format_model.utils [INFO] - Processed /Users/gwang/github/onnxruntime3/onnxruntime/python/tools/quantization/notebooks/imagenet_v2/mobilenet_v2_float.basic.ort
2021-06-03 18:27:58,616 ort_format_model.utils [INFO] - Processed /Users/gwang/github/onnxruntime3/onnxruntime/python/tools/quantization/notebooks/imagenet_v2/mobilenet_v2.uint8.basic.ort
2021-06-03 18:27:58,637 ort_format_model.utils [INFO] - Processed /Users/gwang/github/onnxruntime3/onnxruntime/python/tools/quantization/notebooks/imagenet_v2/mobilenet_v2_float-opt.basic.ort
2021-06-03 18:27:58,675 ort_format_model.utils [INFO] - Processed /Users/gwang/github/onnxruntime3/onnxruntime/python/tools/quantization/notebooks/imagenet_v2/augmented_model.basic.ort
2021-06-03 18:27:58,677 ort_format_model.utils [INFO] - Created config in /Users/gwang/github/onnxruntime3/onnxruntime/python/tools/quantization/notebooks/imagenet_v2/required_operators.config


Please find the following converted models in the same directory,
* mobilenet_v2_float.ort
* mobilenet_v2_uint8.ort
Please note, there are temporary ONNX model files generated by the quantization process, which are converted to ort format as well, please ignore these files.